In [ ]:
!pip install cohere requests beautifulsoup4 lxml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.7/249.7 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00


In [ ]:
import cohere
import requests
import json
from bs4 import BeautifulSoup

co = cohere.Client(api_key="8660B9IAHOrAU4Fma0C9KN0cC6gw9An4tEI2c6aV")

In [ ]:
with open('/content/queries.json', 'r') as f:
    data = json.load(f)

def extract_patient_data(n):
    try:
        query = data[n]
        # Extract relevant fields from the JSON object
        patient_context = query["profile_context"]
        latest_query = "\n".join([f"{q['role']}: {q['content']}" for q in query['latest_query']])
        ideal_response = query["ideal_response"]
        diet_chart = patient_context["diet_chart"]

        # Extract and format notes if available
        try:
            notes_html = diet_chart.get("notes", "")
            soup = BeautifulSoup(notes_html, "lxml")
            formatted_notes = soup.get_text(separator="\n").strip()
            diet_chart["notes"] = formatted_notes
        except:
            pass

        # Collect meal information by day
        meals_info = []
        meals_by_days = diet_chart.get("meals_by_days", [])
        for day in meals_by_days:
            day_info = {"Day": day['order'], "Meals": []}
            meals = day.get("meals", [])
            for meal in meals:
                meal_details = {
                    "Meal ID": meal['id'],
                    "Meal Name": meal['name'],
                    "Meal Timings": meal['timings'],
                    "Meal Options": []
                }
                meal_options = meal.get("meal_options", [])
                for option in meal_options:
                    option_details = {
                        "Option ID": option['id'],
                        "Option Notes": option['notes'],
                        "Food Items": [
                            {
                                "Food ID": food_item['food_id'],
                                "Measure ID": food_item['food_measure_id']
                            }
                            for food_item in option.get("meal_option_food_items", [])
                        ]
                    }
                    meal_details["Meal Options"].append(option_details)
                day_info["Meals"].append(meal_details)
            meals_info.append(day_info)

        return {
            "patient_context": patient_context,
            "latest_query": latest_query,
            "ideal_response": ideal_response,
            "diet_chart": diet_chart,
            "meals_info": meals_info
        }
    except Exception as e:
        raise e


In [ ]:
def suggest_meal(patient_index, custom_query):
    # Extract patient data
    patient_data = extract_patient_data(patient_index)

    # Construct the input for Cohere model based on the custom query and extracted data
    input_text = (
        f"Patient Dietary Information:\n"
        f"{patient_data['latest_query']}\n\n"
        f"Dietary Notes:\n"
        f"{patient_data['diet_chart'].get('notes', 'No specific notes')}\n\n"
        f"Meals by Day:\n"
    )
    for day in patient_data["meals_info"]:
        input_text += f"\nDay {day['Day']}:\n"
        for meal in day["Meals"]:
            input_text += f"  {meal['Meal Name']} at {meal['Meal Timings']}:\n"
            for option in meal["Meal Options"]:
                input_text += f"    Option Notes: {option['Option Notes']}\n"
                input_text += "      Food Items:\n"
                for food in option["Food Items"]:
                    input_text += f"        Food ID: {food['Food ID']}, Measure ID: {food['Measure ID']}\n"

    # Append custom query for personalized meal suggestion
    prompt = f"{input_text}\n\nUser Query: {custom_query}\n\nGenerate a detailed and formatted meal suggestion based on the query and dietary data provided."

    # Use Cohere API to generate a response
    response = co.generate(
        model='command-xlarge-nightly',
        prompt=prompt,
        max_tokens=500,
        temperature=0.7,
        k=0,
        p=0.75,
        stop_sequences=["--"]
    )

    # Clean up and format the output
    suggestion_text = response.generations[0].text.strip()

    # Print in a clean, formatted style
    print("Meal Suggestion for PCOS:\n")
    print("High-Protein Breakfast Option for PCOS\n")
    print(suggestion_text.replace("**", "").replace("*", "").strip())


In [ ]:
# Get user input for a custom meal query
patient_index = 0  # Specify the index of the patient profile
custom_query = input("Enter your meal-related question: ")

# Generate a meal suggestion based on the user's custom query
suggest_meal(patient_index, custom_query)

Enter your meal-related question: What can be a high protein pcos meal?
Meal Suggestion for PCOS:

High-Protein Breakfast Option for PCOS

Here is a high-protein meal suggestion tailored for PCOS, incorporating the dietary notes you provided:

Day 1:

Early Morning (06:30 AM):
- Methi Water: Soak 1 teaspoon of methi seeds in a glass of water overnight, boil in the morning, and strain. This helps reduce bloating and excess water weight.

Breakfast (08:30 AM):
- Poha with Vegetables: Cook 1 cup of poha with a generous portion of chopped vegetables. You can add onions, carrots, peas, and green chilies.
- Milk: Have 1 cup of milk without sugar after 25 minutes.

Mid-Meal (11:00 AM):
- Apple with Seeds: Enjoy an apple sprinkled with a mix of sunflower, sesame, flax, and pumpkin seeds. These seeds support hormonal balance in PCOS.

Lunch (01:30 PM):
- Oats Roti: Mix 50% oats atta with 50% regular atta to make rotis.
- Vegetable Curry: Prepare a curry with a variety of vegetables like carrots

In [ ]:
# Get user input for a custom meal query
patient_index = 0  # Specify the index of the patient profile
custom_query = input("Enter your meal-related question: ")

# Generate a meal suggestion based on the user's custom query
suggest_meal(patient_index, custom_query)

Enter your meal-related question: can I eat pizza?
Meal Suggestion for PCOS:

High-Protein Breakfast Option for PCOS

Meal Suggestion:

Lunch:
- Pizza with a twist:
   - Prepare a whole wheat thin crust pizza base.
   - Top with a tomato-based sauce, avoiding excessive salt.
   - Add a generous amount of green leafy vegetables like spinach, kale, or arugula.
   - Sprinkle with a mix of sunflower and pumpkin seeds for a crunchy texture and extra nutrition.
   - Use low-fat cheese or a plant-based cheese alternative to reduce saturated fat.
   - Bake or grill until the crust is crispy and the cheese is melted.

Side Dish:
- Fresh Green Salad:
   - Combine a variety of green leafy vegetables like lettuce, spinach, and rocket leaves.
   - Add sliced cucumber, cherry tomatoes, and bell peppers for extra color and crunch.
   - Dress with a simple vinaigrette made with olive oil, balsamic vinegar, and a pinch of salt and pepper.

Snack:
- Fruit Salad with Seeds:
   - Chop your favorite fruits